<a href="https://colab.research.google.com/github/Laughing-Bulls/twitter/blob/main/Creating_calculate_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Load the packages required

!pip install pyspark

from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import HashingTF
from pyspark import SparkConf, SparkContext
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import split, regexp_replace
from numpy import array
import numpy as np
import pandas as pd
from pyspark.sql.types import ArrayType, StringType

In [3]:
# Boilerplate Spark stuff:
conf = SparkConf().setMaster("local").setAppName("SparkDecisionTree")
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

In [4]:
# Train model
train = spark.read.csv("processed_training_tweets_SMALL.csv", inferSchema=True, header=True)
train = train.withColumn('words',split(regexp_replace(train["words"], '\[|\]',''),',').cast('array<string>'))
hashTF = HashingTF(inputCol="words", outputCol="numerical")
num_train= hashTF.transform(train).select('score', 'words', 'numerical')
naive_bayes = NaiveBayes(labelCol = "score", featuresCol="numerical", smoothing=1.0, modelType="multinomial").fit(num_train)

In [43]:
tweet_words = ['this', 'is', 'a', 'processed', 'tweet']

column = spark.createDataFrame([tweet_words], ArrayType(StringType())).toDF("words")
column.show(n=5)

hashTF = HashingTF(inputCol="words", outputCol="numerical")
num_column= hashTF.transform(column).select('words', 'numerical')

num_column.show(n=5)

pred_naive_bayes = naive_bayes.transform(num_column)
results_naive_bayes = pred_naive_bayes.select("prediction").replace(1.0, 4.0)
results_naive_bayes.show(n=5)

results = results_naive_bayes.toPandas()
prediction = results.iloc[0,0]
prediction

print(tweet_words, prediction)



+--------------------+
|               words|
+--------------------+
|[this, is, a, pro...|
+--------------------+

+--------------------+--------------------+
|               words|           numerical|
+--------------------+--------------------+
|[this, is, a, pro...|(262144,[106841,1...|
+--------------------+--------------------+

+----------+
|prediction|
+----------+
|       4.0|
+----------+

['this', 'is', 'a', 'processed', 'tweet'] 4.0
